In [3]:
from util_model import FeedForward_DNN, correlation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle
import os
import copy
import tensorflow as tf
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
import time
import collections
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

In [4]:
folder='Mode'
if folder=='Mode':
    data_varlist=['BUS_HIGH','BIKE_HIGH','CAR_HIGH','RIDESHARE_NEW']
    name_corelist=['Bus','Bike','Car','Rideshare']

    Z_var_list=['HH_RACE_MINORITY','FEMALE','POVERTY','MEDCOND_NEW', 'RURAL']
    Z_note_list=['Race','Female','Poverty','Medcond','Rural']
    
else:
    data_varlist=['PLACE','PRICE','2WRK_HOME_NEW','2WKRMHM_NEW']
    name_corelist=['TB','GPI','WFH','WFHO']

    Z_var_list=['HH_RACE_MINORITY','R_SEX_IMP_NEW','POVERTY','MEDCOND_NEW', 'URBRUR_NEW']
    Z_note_list=['Race','Gender','Poverty','Medcond','Urban']


# Fairness measurement

In [4]:
tf.set_random_seed(0)

FP=False

method='cor_soft'#'cor_soft_FP'
datatype='Original'
num_models = 3
n_splits=5

layer=3
learning_rate=1e-4
n_epoch=5000
n_mini_batch=500000

if layer==3:
    model_path ='model_3lyr_'
elif layer==0:
    model_path ='model_base_'


num_alt=2
INCLUDE_VAL_SET = False
INCLUDE_RAW_SET = False

# DIR = 'results/' + folder+'/'+run_name + '/'

lam_list=[0]

start_time = time.time()


for k in zip(data_varlist,name_corelist):
    run_name =model_path+k[1]#e.g.model_base_Bus
    if folder=='Mode':
        with open('data/NHTS_balanced/'+k[0]+'_2size_W.pkl', 'rb') as f:
            input_data = pickle.load(f)
    else:
        with open('data/NHTS_unbalanced/NHTS_'+k[0]+'_W.pkl', 'rb') as f:
            input_data = pickle.load(f)
    X_all_raw=input_data['X_all_raw']
    Y_all=input_data['Y_all']
    
    X_all_raw['FEMALE']=1-X_all_raw['R_SEX_IMP_NEW']
    X_all_raw['RURAL']=1-X_all_raw['URBRUR_NEW']
    
    X_all_raw=X_all_raw.drop(['R_SEX_IMP_NEW','URBRUR_NEW'],axis=1)
    
    for a in zip(Z_var_list,Z_note_list):
        Z_var=a[0]
        DIR = 'results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'/'
        df = []
        for e in range(len(lam_list)):
            p_dic={}
            p_dic['train_accuracy']=[]
            p_dic['test_accuracy']=[]
            p_dic['train_cm0']=[]
            p_dic['train_cm1']=[]
            p_dic['test_cm0']=[]
            p_dic['test_cm1']=[]
            p_dic['train_FNR_gap']=[]
            p_dic['test_FNR_gap']=[]
            p_dic['train_FPR_gap']=[]
            p_dic['test_FPR_gap']=[]

            p_dic['TRAINING_PROCESS']={}
            p_dic['TRAINING_PROCESS']['best_iter']=[]
            p_dic['TRAINING_PROCESS']['final_iter']=[]
            p_dic['TRAINING_PROCESS']['accuracy']={}
            p_dic['TRAINING_PROCESS']['cost1']={}
            p_dic['TRAINING_PROCESS']['cost2']={}
            p_dic['TRAINING_PROCESS']['accuracy']['training_set']=[]
            p_dic['TRAINING_PROCESS']['accuracy']['testing_set']=[]
            p_dic['TRAINING_PROCESS']['cost1']['training_set']=[]
            p_dic['TRAINING_PROCESS']['cost1']['testing_set']=[]
            p_dic['TRAINING_PROCESS']['cost2']['training_set']=[]
            p_dic['TRAINING_PROCESS']['cost2']['testing_set']=[]
            


            for i in range(num_models):
                kf = KFold(n_splits, shuffle=True,random_state=0)

                #final results
                accuracy_train_list = []
                accuracy_test_list = []

                train_cm0_trial=[]
                train_cm1_trial=[]
                test_cm0_trial=[]
                test_cm1_trial=[]

                train_FNR_gap_list=[]
                test_FNR_gap_list=[]
                train_FPR_gap_list=[]
                test_FPR_gap_list=[]

                #training process
                best_iter_trial=[]
                final_iter_trial=[]
                accuracy_training_prcoess_trial=[]
                accuracy_testing_prcoess_trial=[]
                cost1_training_prcoess_trial=[]
                cost1_testing_prcoess_trial=[]
                cost2_training_prcoess_trial=[]
                cost2_testing_prcoess_trial=[]

                con=0
                for train_idx, test_idx in kf.split(X_all_raw,Y_all):
                    con+=1
                    MODEL_NAME = 'lambda'+str(lam_list[e])+'_trial' + str(i)+'_fold'+str(con)

                    X_train_raw = X_all_raw.iloc[train_idx,:]
                    Y_train = Y_all[train_idx]

                    X_test_raw = X_all_raw.iloc[test_idx,:]
                    Y_test = Y_all[test_idx]
                    
                    X_train=X_train_raw.loc[:, X_train_raw.columns != 'WTPERFIN']
                    X_test=X_test_raw.loc[:, X_test_raw.columns != 'WTPERFIN']
                    W_train=X_train_raw['WTPERFIN']
                    W_test=X_test_raw['WTPERFIN']
                    
                    
                    N_bootstrap_sample = len(X_train_raw)
                    F_DNN = FeedForward_DNN(num_alt,MODEL_NAME,INCLUDE_VAL_SET,INCLUDE_RAW_SET, DIR)
                    
                    F_DNN.load_data(Z_var)
                    F_DNN.init_hyperparameter(lam_list[e],lr=learning_rate,lyr=layer,n_epoch=n_epoch,n_mini_batch=n_mini_batch) # could change the hyperparameter here by using F_DNN.change_hyperparameter(new)
                    F_DNN.bootstrap_data(N_bootstrap_sample)
                    F_DNN.build_model(method)
                    F_DNN.train_model()
                    F_DNN.evaluate_model()
                     
                    print(" ")
                    print('Y:',k[1],'; Z:',a[1],' ',MODEL_NAME,' completed')
                    print('accuracy_train: ',F_DNN.accuracy_train,' ;accuracy_test: ',F_DNN.accuracy_test)


                    #save results
                    accuracy_train_list.append(F_DNN.accuracy_train)
                    accuracy_test_list.append(F_DNN.accuracy_test)

                    train_cm0=F_DNN.confusion_matrix_z0_train.ravel()
                    train_cm1=F_DNN.confusion_matrix_z1_train.ravel()
                    train_cm0_trial.append(train_cm0)
                    train_cm1_trial.append(train_cm1)

                    test_cm0=F_DNN.confusion_matrix_z0_test.ravel()
                    test_cm1=F_DNN.confusion_matrix_z1_test.ravel()
                    test_cm0_trial.append(test_cm0)
                    test_cm1_trial.append(test_cm1)

                    train_FNR0=train_cm0[2]/(train_cm0[2]+train_cm0[3])
                    train_FNR1=train_cm1[2]/(train_cm1[2]+train_cm1[3])
                    train_FNR_gap=train_FNR0-train_FNR1

                    test_FNR0=test_cm0[2]/(test_cm0[2]+test_cm0[3])
                    test_FNR1=test_cm1[2]/(test_cm1[2]+test_cm1[3])
                    test_FNR_gap=test_FNR0-test_FNR1
                    
                    train_FPR0=train_cm0[1]/(train_cm0[0]+train_cm0[1])
                    train_FPR1=train_cm1[1]/(train_cm1[0]+train_cm1[1])
                    train_FPR_gap=train_FPR0-train_FPR1

                    test_FPR0=test_cm0[1]/(test_cm0[0]+test_cm0[1])
                    test_FPR1=test_cm1[1]/(test_cm1[0]+test_cm1[1])
                    test_FPR_gap=test_FPR0-test_FPR1

                    train_FNR_gap_list.append(train_FNR_gap)
                    test_FNR_gap_list.append(test_FNR_gap)
                    train_FPR_gap_list.append(train_FPR_gap)
                    test_FPR_gap_list.append(test_FPR_gap)
                    if not FP:
                        print('train_FNR_gap: ',train_FNR_gap,' ;test_FNR_gap: ',test_FNR_gap)
                    else:
                        print('train_FPR_gap: ',train_FPR_gap,' ;test_FPR_gap: ',test_FPR_gap)
                    print('best_iter: ',F_DNN.best_iter)
                    time_dur=round(time.time() - start_time, 3)
                    print("--- %s seconds ---" % round((time.time() - start_time),4))
                    print(" ")

                    #save process
                    best_iter_trial.append(F_DNN.best_iter)
                    final_iter_trial.append(F_DNN.final_iter)
                    accuracy_training_prcoess_trial.append(F_DNN.train_accuracy_list)
                    accuracy_testing_prcoess_trial.append(F_DNN.test_accuracy_list)
                    cost1_training_prcoess_trial.append(F_DNN.train_cost1_list)
                    cost1_testing_prcoess_trial.append(F_DNN.test_cost1_list)
                    cost2_training_prcoess_trial.append(F_DNN.train_cost2_list)
                    cost2_testing_prcoess_trial.append(F_DNN.test_cost2_list)

    #                     print(n_samples)
                    df.append([lam_list[e], i,con,F_DNN.h['M'], learning_rate, F_DNN.h['n_mini_batch'],F_DNN.best_iter,F_DNN.final_iter,\
                    F_DNN.accuracy_train, F_DNN.accuracy_test,\
                    train_FNR_gap,test_FNR_gap,train_FPR_gap,test_FPR_gap])


                p_dic['train_accuracy'].append(accuracy_train_list)
                p_dic['test_accuracy'].append(accuracy_test_list)
                p_dic['train_cm0'].append(train_cm0_trial)
                p_dic['train_cm1'].append(train_cm1_trial)
                p_dic['test_cm0'].append(test_cm0_trial)
                p_dic['test_cm1'].append(test_cm1_trial)
                p_dic['train_FNR_gap'].append(train_FNR_gap_list)
                p_dic['test_FNR_gap'].append(test_FNR_gap_list)
                p_dic['train_FPR_gap'].append(train_FPR_gap_list)
                p_dic['test_FPR_gap'].append(test_FPR_gap_list)

                p_dic['TRAINING_PROCESS']['best_iter'].append(best_iter_trial)
                p_dic['TRAINING_PROCESS']['final_iter'].append(final_iter_trial)
                p_dic['TRAINING_PROCESS']['accuracy']['training_set'].append(accuracy_training_prcoess_trial)
                p_dic['TRAINING_PROCESS']['accuracy']['testing_set'].append(accuracy_testing_prcoess_trial)
                p_dic['TRAINING_PROCESS']['cost1']['training_set'].append(cost1_training_prcoess_trial)
                p_dic['TRAINING_PROCESS']['cost1']['testing_set'].append(cost1_testing_prcoess_trial)
                p_dic['TRAINING_PROCESS']['cost2']['training_set'].append(cost2_training_prcoess_trial)
                p_dic['TRAINING_PROCESS']['cost2']['testing_set'].append(cost2_testing_prcoess_trial)

                with open('results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'.pkl', 'wb') as f:
                    pickle.dump(p_dic, f)


                df = pd.DataFrame(df, columns = ['lambda','trial','fold','#layers','#lr','batch_size','best_iter','final_iter',\
                                                 'accuracy_train','accuracy_test','FNR_gap_train','FNR_gap_test',\
                                                'FPR_gap_train','FPR_gap_test'])
                df.to_csv('results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'.csv')

# Regularization Section

In [ ]:
folder='Mode'
if folder=='Mode':
    data_varlist=['RIDESHARE_NEW']
    name_corelist=['Rideshare']
    
    Z_var_list=['RURAL']
    Z_note_list=['Rural']
    
else:
    data_varlist=['PLACE']
    name_corelist=['TB']

    Z_var_list=['HH_RACE_MINORITY']
    Z_note_list=['Race']

In [ ]:
tf.set_random_seed(0)

FP=False

if folder=='WFH_Group':
    method='cor_soft_FP'
else:
    method='cor_soft'
datatype='Original'
num_models = 1
n_splits=5

layer=0
learning_rate=1e-3
n_epoch=3000
n_mini_batch=500000

if layer==3:
    model_path ='model_3lyr_'
elif layer==0:
    model_path ='model_base_'


num_alt=2
INCLUDE_VAL_SET = False
INCLUDE_RAW_SET = False

# DIR = 'results/' + folder+'/'+run_name + '/'

lam_list=[0.2,0.4,0.6,0.8]


start_time = time.time()


    
for k in zip(data_varlist,name_corelist):
    run_name =model_path+k[1]#e.g.model_base_Bus
    if folder=='Mode':
        with open('data/NHTS_balanced/'+k[0]+'_2size_W.pkl', 'rb') as f:
            input_data = pickle.load(f)
    else:
        with open('data/NHTS_unbalanced/NHTS_'+k[0]+'_W.pkl', 'rb') as f:
            input_data = pickle.load(f)
    X_all_raw=input_data['X_all_raw']
    Y_all=input_data['Y_all']
    
    X_all_raw['FEMALE']=1-X_all_raw['R_SEX_IMP_NEW']
    X_all_raw['RURAL']=1-X_all_raw['URBRUR_NEW']
    
    X_all_raw=X_all_raw.drop(['R_SEX_IMP_NEW','URBRUR_NEW'],axis=1)
    
    for a in zip(Z_var_list,Z_note_list):
        Z_var=a[0]
        DIR = 'results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'/'
        df = []
        for e in range(len(lam_list)):
            p_dic={}
            p_dic['train_accuracy']=[]
            p_dic['test_accuracy']=[]
            p_dic['train_cm0']=[]
            p_dic['train_cm1']=[]
            p_dic['test_cm0']=[]
            p_dic['test_cm1']=[]
            p_dic['train_FNR_gap']=[]
            p_dic['test_FNR_gap']=[]
            p_dic['train_FPR_gap']=[]
            p_dic['test_FPR_gap']=[]

            p_dic['TRAINING_PROCESS']={}
            p_dic['TRAINING_PROCESS']['best_iter']=[]
            p_dic['TRAINING_PROCESS']['final_iter']=[]
            p_dic['TRAINING_PROCESS']['accuracy']={}
            p_dic['TRAINING_PROCESS']['cost1']={}
            p_dic['TRAINING_PROCESS']['cost2']={}
            p_dic['TRAINING_PROCESS']['accuracy']['training_set']=[]
            p_dic['TRAINING_PROCESS']['accuracy']['testing_set']=[]
            p_dic['TRAINING_PROCESS']['cost1']['training_set']=[]
            p_dic['TRAINING_PROCESS']['cost1']['testing_set']=[]
            p_dic['TRAINING_PROCESS']['cost2']['training_set']=[]
            p_dic['TRAINING_PROCESS']['cost2']['testing_set']=[]
            


            for i in range(num_models):
                kf = KFold(n_splits, shuffle=True,random_state=0)

                #final results
                accuracy_train_list = []
                accuracy_test_list = []

                train_cm0_trial=[]
                train_cm1_trial=[]
                test_cm0_trial=[]
                test_cm1_trial=[]

                train_FNR_gap_list=[]
                test_FNR_gap_list=[]
                train_FPR_gap_list=[]
                test_FPR_gap_list=[]

                #training process
                best_iter_trial=[]
                final_iter_trial=[]
                accuracy_training_prcoess_trial=[]
                accuracy_testing_prcoess_trial=[]
                cost1_training_prcoess_trial=[]
                cost1_testing_prcoess_trial=[]
                cost2_training_prcoess_trial=[]
                cost2_testing_prcoess_trial=[]

                con=0
                for train_idx, test_idx in kf.split(X_all_raw,Y_all):
                    con+=1
                    MODEL_NAME = 'lambda'+str(lam_list[e])+'_trial' + str(i)+'_fold'+str(con)

                    X_train_raw = X_all_raw.iloc[train_idx,:]
                    Y_train = Y_all[train_idx]

                    X_test_raw = X_all_raw.iloc[test_idx,:]
                    Y_test = Y_all[test_idx]
                    
                    X_train=X_train_raw.loc[:, X_train_raw.columns != 'WTPERFIN']
                    X_test=X_test_raw.loc[:, X_test_raw.columns != 'WTPERFIN']
                    W_train=X_train_raw['WTPERFIN']
                    W_test=X_test_raw['WTPERFIN']
                    
                    
                    N_bootstrap_sample = len(X_train_raw)
                    F_DNN = FeedForward_DNN(num_alt,MODEL_NAME,INCLUDE_VAL_SET,INCLUDE_RAW_SET, DIR)
                    
                    F_DNN.load_data(Z_var)
                    F_DNN.init_hyperparameter(lam_list[e],lr=learning_rate,lyr=layer,n_epoch=n_epoch,n_mini_batch=n_mini_batch) # could change the hyperparameter here by using F_DNN.change_hyperparameter(new)
                    F_DNN.bootstrap_data(N_bootstrap_sample)
                    F_DNN.build_model(method)
                    F_DNN.train_model()
                    F_DNN.evaluate_model()
                     
                    print(" ")
                    print('Y:',k[1],'; Z:',a[1],' ',MODEL_NAME,' completed')
                    print('accuracy_train: ',F_DNN.accuracy_train,' ;accuracy_test: ',F_DNN.accuracy_test)


                    #save results
                    accuracy_train_list.append(F_DNN.accuracy_train)
                    accuracy_test_list.append(F_DNN.accuracy_test)

                    train_cm0=F_DNN.confusion_matrix_z0_train.ravel()
                    train_cm1=F_DNN.confusion_matrix_z1_train.ravel()
                    train_cm0_trial.append(train_cm0)
                    train_cm1_trial.append(train_cm1)

                    test_cm0=F_DNN.confusion_matrix_z0_test.ravel()
                    test_cm1=F_DNN.confusion_matrix_z1_test.ravel()
                    test_cm0_trial.append(test_cm0)
                    test_cm1_trial.append(test_cm1)

                    train_FNR0=train_cm0[2]/(train_cm0[2]+train_cm0[3])
                    train_FNR1=train_cm1[2]/(train_cm1[2]+train_cm1[3])
                    train_FNR_gap=train_FNR0-train_FNR1

                    test_FNR0=test_cm0[2]/(test_cm0[2]+test_cm0[3])
                    test_FNR1=test_cm1[2]/(test_cm1[2]+test_cm1[3])
                    test_FNR_gap=test_FNR0-test_FNR1
                    
                    train_FPR0=train_cm0[1]/(train_cm0[0]+train_cm0[1])
                    train_FPR1=train_cm1[1]/(train_cm1[0]+train_cm1[1])
                    train_FPR_gap=train_FPR0-train_FPR1

                    test_FPR0=test_cm0[1]/(test_cm0[0]+test_cm0[1])
                    test_FPR1=test_cm1[1]/(test_cm1[0]+test_cm1[1])
                    test_FPR_gap=test_FPR0-test_FPR1

                    train_FNR_gap_list.append(train_FNR_gap)
                    test_FNR_gap_list.append(test_FNR_gap)
                    train_FPR_gap_list.append(train_FPR_gap)
                    test_FPR_gap_list.append(test_FPR_gap)

                    print('train_FNR_gap: ',train_FNR_gap,' ;test_FNR_gap: ',test_FNR_gap)
                    print('train_FPR_gap: ',train_FPR_gap,' ;test_FPR_gap: ',test_FPR_gap)
                    if con>1:
                        print('best_iter: ',F_DNN.best_iter)
                    time_dur=round(time.time() - start_time, 3)
                    print("--- %s seconds ---" % round((time.time() - start_time),4))
                    print(" ")

                    #save process
                    
                    best_iter_trial.append(F_DNN.best_iter)
                    final_iter_trial.append(F_DNN.final_iter)
                    accuracy_training_prcoess_trial.append(F_DNN.train_accuracy_list)
                    accuracy_testing_prcoess_trial.append(F_DNN.test_accuracy_list)
                    cost1_training_prcoess_trial.append(F_DNN.train_cost1_list)
                    cost1_testing_prcoess_trial.append(F_DNN.test_cost1_list)
                    cost2_training_prcoess_trial.append(F_DNN.train_cost2_list)
                    cost2_testing_prcoess_trial.append(F_DNN.test_cost2_list)

                    df.append([lam_list[e], i,con,F_DNN.h['M'], learning_rate, F_DNN.h['n_mini_batch'],F_DNN.best_iter,F_DNN.final_iter,\
                    F_DNN.accuracy_train, F_DNN.accuracy_test,\
                    train_FNR_gap,test_FNR_gap,train_FPR_gap,test_FPR_gap])


                p_dic['train_accuracy'].append(accuracy_train_list)
                p_dic['test_accuracy'].append(accuracy_test_list)
                p_dic['train_cm0'].append(train_cm0_trial)
                p_dic['train_cm1'].append(train_cm1_trial)
                p_dic['test_cm0'].append(test_cm0_trial)
                p_dic['test_cm1'].append(test_cm1_trial)
                p_dic['train_FNR_gap'].append(train_FNR_gap_list)
                p_dic['test_FNR_gap'].append(test_FNR_gap_list)
                p_dic['train_FPR_gap'].append(train_FPR_gap_list)
                p_dic['test_FPR_gap'].append(test_FPR_gap_list)

                p_dic['TRAINING_PROCESS']['best_iter'].append(best_iter_trial)
                p_dic['TRAINING_PROCESS']['final_iter'].append(final_iter_trial)
                p_dic['TRAINING_PROCESS']['accuracy']['training_set'].append(accuracy_training_prcoess_trial)
                p_dic['TRAINING_PROCESS']['accuracy']['testing_set'].append(accuracy_testing_prcoess_trial)
                p_dic['TRAINING_PROCESS']['cost1']['training_set'].append(cost1_training_prcoess_trial)
                p_dic['TRAINING_PROCESS']['cost1']['testing_set'].append(cost1_testing_prcoess_trial)
                p_dic['TRAINING_PROCESS']['cost2']['training_set'].append(cost2_training_prcoess_trial)
                p_dic['TRAINING_PROCESS']['cost2']['testing_set'].append(cost2_testing_prcoess_trial)

                with open('results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'_lam'+str(lam_list[e])+'.pkl', 'wb') as f:
                    pickle.dump(p_dic, f)


            df = pd.DataFrame(df, columns = ['lambda','trial','fold','#layers','#lr','batch_size','best_iter','final_iter',\
                                             'accuracy_train','accuracy_test','FNR_gap_train','FNR_gap_test',\
                                            'FPR_gap_train','FPR_gap_test'])
            df.to_csv('results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'_lam'+str(lam_list[e])+'.csv')

# Restore trained models

In [8]:

folder='Mode'
if folder=='Mode':
    data_varlist=['BUS_HIGH','BIKE_HIGH','CAR_HIGH','RIDESHARE_NEW']
    name_corelist=['Bus','Bike','Car','Rideshare']

    Z_var_list=['HH_RACE_MINORITY','FEMALE','POVERTY','MEDCOND_NEW', 'RURAL']
    Z_note_list=['Race','Female','Poverty','Medcond','Rural']
    
else:
    data_varlist=['PLACE','PRICE','2WRK_HOME_NEW','2WKRMHM_NEW']
    name_corelist=['TB','GPI','WFH','WFHO']

    Z_var_list=['HH_RACE_MINORITY','R_SEX_IMP_NEW','POVERTY','MEDCOND_NEW', 'URBRUR_NEW']
    Z_note_list=['Race','Gender','Poverty','Medcond','Urban']


In [5]:
tf.set_random_seed(0)


method='cor_soft'
datatype='Original'
num_models = 3
n_splits=5

layer=3
learning_rate=1e-4
n_epoch=5000
n_mini_batch=500000

if layer==3:
    model_path ='model_3lyr_'
elif layer==0:
    model_path ='model_base_'


num_alt=2
INCLUDE_VAL_SET = False
INCLUDE_RAW_SET = False

# DIR = 'results/' + folder+'/'+run_name + '/'
num_training_samples = None


lam_list=[0]

start_time = time.time()


    
for k in zip(data_varlist,name_corelist):
    run_name =model_path+k[1]#e.g.model_base_Bus
    if folder=='Mode':
        with open('data/NHTS_balanced/'+k[0]+'_2size_W.pkl', 'rb') as f:
            input_data = pickle.load(f)
    else:
        with open('data/NHTS_unbalanced/NHTS_'+k[0]+'_W.pkl', 'rb') as f:
            input_data = pickle.load(f)
    X_all_raw=input_data['X_all_raw']
    Y_all=input_data['Y_all']
    
    X_all_raw['FEMALE']=1-X_all_raw['R_SEX_IMP_NEW']
    X_all_raw['RURAL']=1-X_all_raw['URBRUR_NEW']
    
    X_all_raw=X_all_raw.drop(['R_SEX_IMP_NEW','URBRUR_NEW'],axis=1)
    
    for a in zip(Z_var_list,Z_note_list):
        Z_var=a[0]
        DIR = 'results/' + folder+'/'+ datatype+'/'+ 'Race'+'/'+run_name+'/'
        df = []
        for e in range(len(lam_list)):
            p_dic={}
            p_dic['train_accuracy']=[]
            p_dic['test_accuracy']=[]
            p_dic['train_cm0']=[]
            p_dic['train_cm1']=[]
            p_dic['test_cm0']=[]
            p_dic['test_cm1']=[]
            p_dic['train_FNR_gap']=[]
            p_dic['test_FNR_gap']=[]
            p_dic['train_FPR_gap']=[]
            p_dic['test_FPR_gap']=[]

            p_dic['TRAINING_PROCESS']={}
            p_dic['TRAINING_PROCESS']['best_iter']=[]
            p_dic['TRAINING_PROCESS']['final_iter']=[]
            p_dic['TRAINING_PROCESS']['accuracy']={}
            p_dic['TRAINING_PROCESS']['cost1']={}
            p_dic['TRAINING_PROCESS']['cost2']={}
            p_dic['TRAINING_PROCESS']['accuracy']['training_set']=[]
            p_dic['TRAINING_PROCESS']['accuracy']['testing_set']=[]
            p_dic['TRAINING_PROCESS']['cost1']['training_set']=[]
            p_dic['TRAINING_PROCESS']['cost1']['testing_set']=[]
            p_dic['TRAINING_PROCESS']['cost2']['training_set']=[]
            p_dic['TRAINING_PROCESS']['cost2']['testing_set']=[]
                        
            with open('results/' + folder+'/'+ datatype+'/'+ 'Race'+'/'+run_name+'.pkl', 'rb') as f:
                p_dic_ref = pickle.load(f)

            p_dic['TRAINING_PROCESS']=p_dic_ref['TRAINING_PROCESS']


            for i in range(num_models):
                kf = KFold(n_splits, shuffle=True,random_state=0)

                #final results
                accuracy_train_list = []
                accuracy_test_list = []

                train_cm0_trial=[]
                train_cm1_trial=[]
                test_cm0_trial=[]
                test_cm1_trial=[]

                train_FNR_gap_list=[]
                test_FNR_gap_list=[]
                train_FPR_gap_list=[]
                test_FPR_gap_list=[]

                #training process
                best_iter_trial=[]
                final_iter_trial=[]
                accuracy_training_prcoess_trial=[]
                accuracy_testing_prcoess_trial=[]
                cost1_training_prcoess_trial=[]
                cost1_testing_prcoess_trial=[]
                cost2_training_prcoess_trial=[]
                cost2_testing_prcoess_trial=[]

                con=0
                for train_idx, test_idx in kf.split(X_all_raw,Y_all):
                    con+=1
                    MODEL_NAME = 'lambda'+str(lam_list[e])+'_trial' + str(i)+'_fold'+str(con)

                    X_train_raw = X_all_raw.iloc[train_idx,:]
                    Y_train = Y_all[train_idx]

                    X_test_raw = X_all_raw.iloc[test_idx,:]
                    Y_test = Y_all[test_idx]
                    
                    X_train=X_train_raw.loc[:, X_train_raw.columns != 'WTPERFIN']
                    X_test=X_test_raw.loc[:, X_test_raw.columns != 'WTPERFIN']
                    W_train=X_train_raw['WTPERFIN']
                    W_test=X_test_raw['WTPERFIN']
                    
                    
                    N_bootstrap_sample = len(X_train_raw)
                    F_DNN = FeedForward_DNN(num_alt,MODEL_NAME,INCLUDE_VAL_SET,INCLUDE_RAW_SET, DIR)
                    
                    F_DNN.load_data(Z_var)
                    F_DNN.init_hyperparameter(lam_list[e],lr=learning_rate,lyr=layer,n_epoch=n_epoch,n_mini_batch=n_mini_batch) # could change the hyperparameter here by using F_DNN.change_hyperparameter(new)
                    F_DNN.bootstrap_data(N_bootstrap_sample)
                    F_DNN.build_model(method)
                    F_DNN.evaluate_model()
                     
                    print(" ")
                    print('Y:',k[1],'; Z:',a[1],' ',MODEL_NAME,' completed')
                    print('accuracy_train: ',F_DNN.accuracy_train,' ;accuracy_test: ',F_DNN.accuracy_test)


                    #save results
                    accuracy_train_list.append(F_DNN.accuracy_train)
                    accuracy_test_list.append(F_DNN.accuracy_test)

                    train_cm0=F_DNN.confusion_matrix_z0_train.ravel()
                    train_cm1=F_DNN.confusion_matrix_z1_train.ravel()
                    train_cm0_trial.append(train_cm0)
                    train_cm1_trial.append(train_cm1)

                    test_cm0=F_DNN.confusion_matrix_z0_test.ravel()
                    test_cm1=F_DNN.confusion_matrix_z1_test.ravel()
                    test_cm0_trial.append(test_cm0)
                    test_cm1_trial.append(test_cm1)

                    train_FNR0=train_cm0[2]/(train_cm0[2]+train_cm0[3])
                    train_FNR1=train_cm1[2]/(train_cm1[2]+train_cm1[3])
                    train_FNR_gap=train_FNR0-train_FNR1


                    test_FNR0=test_cm0[2]/(test_cm0[2]+test_cm0[3])
                    test_FNR1=test_cm1[2]/(test_cm1[2]+test_cm1[3])
                    test_FNR_gap=test_FNR0-test_FNR1
                    
                    train_FPR0=train_cm0[1]/(train_cm0[0]+train_cm0[1])
                    train_FPR1=train_cm1[1]/(train_cm1[0]+train_cm1[1])
                    train_FPR_gap=train_FPR0-train_FPR1

                    test_FPR0=test_cm0[1]/(test_cm0[0]+test_cm0[1])
                    test_FPR1=test_cm1[1]/(test_cm1[0]+test_cm1[1])
                    test_FPR_gap=test_FPR0-test_FPR1

                    train_FNR_gap_list.append(train_FNR_gap)
                    test_FNR_gap_list.append(test_FNR_gap)
                    train_FPR_gap_list.append(train_FPR_gap)
                    test_FPR_gap_list.append(test_FPR_gap)

                    print('train_FNR_gap: ',train_FNR_gap,' ;test_FNR_gap: ',test_FNR_gap)
                    print('train_FPR_gap: ',train_FPR_gap,' ;test_FPR_gap: ',test_FPR_gap)
                    time_dur=round(time.time() - start_time, 3)
                    print("--- %s seconds ---" % round((time.time() - start_time),4))
                    print(" ")


                    df.append([lam_list[e], i,con,F_DNN.h['M'], learning_rate, F_DNN.h['n_mini_batch'],\
                    F_DNN.accuracy_train, F_DNN.accuracy_test,\
                    train_FNR_gap,test_FNR_gap,train_FPR_gap,test_FPR_gap])


                p_dic['train_accuracy'].append(accuracy_train_list)
                p_dic['test_accuracy'].append(accuracy_test_list)
                p_dic['train_cm0'].append(train_cm0_trial)
                p_dic['train_cm1'].append(train_cm1_trial)
                p_dic['test_cm0'].append(test_cm0_trial)
                p_dic['test_cm1'].append(test_cm1_trial)
                p_dic['train_FNR_gap'].append(train_FNR_gap_list)
                p_dic['test_FNR_gap'].append(test_FNR_gap_list)
                p_dic['train_FPR_gap'].append(train_FPR_gap_list)
                p_dic['test_FPR_gap'].append(test_FPR_gap_list)

            with open('results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'.pkl', 'wb') as f:
                pickle.dump(p_dic, f)


            df = pd.DataFrame(df, columns = ['lambda','trial','fold','#layers','#lr','batch_size',\
                                             'accuracy_train','accuracy_test','FNR_gap_train','FNR_gap_test','FPR_gap_train','FPR_gap_test'])
            df.to_csv('results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'.csv')